In [1]:
import numpy as np
import pandas as pd
from glob import glob
import os

In [16]:
snr_cat = pd.read_pickle('../dataframes/high_snr_df.pkl')

In [17]:
idx = 1
for i in snr_cat.index:
    slrm_text= ["#!/bin/bash\n#SBATCH -J bfit-{0}\n#SBATCH -p background\n#SBATCH --time=8:00:00\n#SBATCH --mem-per-cpu=32000\n#SBATCH -o bfit{0}-%j.out\n\
#SBATCH -e bfit{0}-%j.err\n\nchmod a+x bestfit.py\n\n\
python bestfit.py {1} {2} {3} &\n\
\nwait\n\nexit 0".format(idx, snr_cat.field[i],snr_cat.id[i], np.round(snr_cat.z[i],4))]
    np.savetxt('../cluster_scripts/bestfit_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
    idx +=1


In [12]:
print(slrm_text[0])

#!/bin/bash
#SBATCH -J bfit-61
#SBATCH -p background
#SBATCH --time=2:00:00
#SBATCH --mem-per-cpu=16000
#SBATCH -o bfit61-%j.out
#SBATCH -e bfit61-%j.err

chmod a+x bestfit.py

python bestfit.py GSD 35774 1.2533 &

wait

exit 0


In [2]:
select = pd.read_pickle('../spec_files/all_section.pkl')
select = select.query('AGN != "AGN" and use == True')

In [11]:
idx = 1
test_ids = [21156, 39170, 41520]
for i in select.index:   
    if select.id[i] in test_ids:
        if os.path.isfile('../data/posteriors/{0}_{1}_zfit_Pz.npy'.format(select.field[i], select.id[i])):
            z,pz = np.load('../data/posteriors/{0}_{1}_zfit_Pz.npy'.format(select.field[i], select.id[i]))
            zfit = z[pz == max(pz)][0] 
        else:
            zfit =  select.zphot[i]

        slrm_text= ["#!/bin/bash\n#SBATCH -J ttst-{0}\n#SBATCH -p background\n#SBATCH -c 8\n\
#SBATCH --time=16:00:00\n#SBATCH --mem=32gb\n#SBATCH -o ttst{0}-%j.out\n#SBATCH -e ttst{0}-%j.err\n\nchmod a+x fullfit-test.py\n\n\
python fullfit-test.py {1} {2} {3:1.4f}\n\
\nwait\n\nexit 0".format(idx, select.field[i], select.id[i], zfit)]
        np.savetxt('../cluster_scripts/fullfit-test_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
        idx +=1   
        

In [10]:
print(slrm_text[0])

#!/bin/bash
#SBATCH -J ttst-3
#SBATCH -p background
#SBATCH -c 8
#SBATCH --time=16:00:00
#SBATCH --mem=32gb
#SBATCH -o ttst3-%j.out
#SBATCH -e ttst3-%j.err

chmod a+x fullfit-test.py

python fullfit-test.py GSD 41520 1.6148

wait

exit 0


In [3]:
idx = 1
for i in select.index:   
    if select.use[i]:
        slrm_text= ["#!/bin/bash\n#SBATCH -J zfit-{0}\n#SBATCH -p background\n#SBATCH -c 8\n\
#SBATCH --time=16:00:00\n#SBATCH --mem=32gb\n#SBATCH -o zfit{0}-%j.out\n#SBATCH -e zfit{0}-%j.err\n\nchmod a+x redshift_fit.py\n\n\
python redshift_fit.py {1} {2} {3:1.4f}\n\
\nwait\n\nexit 0".format(idx, select.field[i], select.id[i], select.zphot[i])]
        np.savetxt('../cluster_scripts/zfit_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
        idx +=1                  

In [4]:
print(slrm_text[0])

#!/bin/bash
#SBATCH -J zfit-166
#SBATCH -p background
#SBATCH -c 8
#SBATCH --time=16:00:00
#SBATCH --mem=32gb
#SBATCH -o zfit166-%j.out
#SBATCH -e zfit166-%j.err

chmod a+x redshift_fit.py

python redshift_fit.py GSD 49072 0.9914

wait

exit 0


In [23]:
idx = 1

errterm = [0.02,0.03,0.04,0.05]
for i in errterm:
    for ii in range(10):   
        slrm_text= ["#!/bin/bash\n#SBATCH -J tfit-{1}\n#SBATCH -p background\n#SBATCH -c 8\n\
#SBATCH --time=16:00:00\n#SBATCH --mem=32gb\n#SBATCH -o tfit{1}-%j.out\n#SBATCH -e tfit{1}-%j.err\n\nchmod a+x fullfit-test.py\n\n\
python fullfit-test.py GND 21156 1.255 {0} {1}\n\
\nwait\n\nexit 0".format(i, idx)]
        np.savetxt('../cluster_scripts/fullfit-test_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
        idx +=1  

In [2]:
flist = glob('../data/posteriors/*zfit*')

fld = [os.path.basename(U).split('_')[0] for U in flist] 
ids = [os.path.basename(U).split('_')[1] for U in flist] 

In [13]:
idx = 1
for i in select.index:   
    if os.path.isfile('../data/posteriors/{0}_{1}_zfit_Pz.npy'.format(select.field[i], select.id[i])):
        z,pz = np.load('../data/posteriors/{0}_{1}_zfit_Pz.npy'.format(select.field[i], select.id[i]))
        zfit = z[pz == max(pz)][0] 
    else:
        zfit =  select.zphot[i]
        
    slrm_text= ["#!/bin/bash\n#SBATCH -J tfit-{0}\n#SBATCH -p background\n#SBATCH -c 8\n\
#SBATCH --time=16:00:00\n#SBATCH --mem=32gb\n#SBATCH -o tfit{0}-%j.out\n#SBATCH -e tfit{0}-%j.err\n\nchmod a+x fullfit.py\n\n\
python fullfit.py {1} {2} {3:1.4f}\n\
\nwait\n\nexit 0".format(idx, select.field[i], select.id[i], zfit)]
    np.savetxt('../cluster_scripts/fullfit_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
    idx +=1  

In [28]:
idx = 1
for i in select.index: 
    if os.path.isfile('../data/posteriors/{0}_{1}_zfit_Pz.npy'.format(select.field[i], select.id[i]))
        z,pz = np.load('../data/posteriors/{0}_{1}_zfit_Pz.npy'.format(select.field[i], select.id[i]))
        zfit = z[pz == max(pz)][0] 
    else:
        zfit =  select.zphot[i]
    slrm_text= ["#!/bin/bash\n#SBATCH -J dfit-{0}\n#SBATCH -p background\n#SBATCH -c 8\n\
#SBATCH --time=16:00:00\n#SBATCH --mem=32gb\n#SBATCH -o dfit{0}-%j.out\n#SBATCH -e dfit{0}-%j.err\n\nchmod a+x delayfit.py\n\n\
python delayfit.py {1} {2} {3:1.4f}\n\
\nwait\n\nexit 0".format(idx, select.field[i], select.id[i], zfit)]
    np.savetxt('../cluster_scripts/delayfit_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
    idx +=1  

In [12]:
print(slrm_text[0])

#!/bin/bash
#SBATCH -J tfit-166
#SBATCH -p background
#SBATCH -c 8
#SBATCH --time=16:00:00
#SBATCH --mem=32gb
#SBATCH -o tfit166-%j.out
#SBATCH -e tfit166-%j.err

chmod a+x fullfit.py

python fullfit.py GSD 49072 1.0630

wait

exit 0


In [13]:
####ALMA Zfit########
gals = [26272, 29257, 38843, 39012, 39364, 40223, 41520, 42113, 42778,
        43007, 44042]

LIMSR = [None, None, None, None, None, [11200,12500], None, [11200,15400], [14000,16500], None, None]

idx = 1

for i in range(len(gals)):
    if LIMSR[i] == None:
        lim1 = 11200
        lim2 = 16500
    else:
        lim1 = LIMSR[i][0]
        lim2 = LIMSR[i][1]
    
    slrm_text= ["#!/bin/bash\n#SBATCH -J afit-{0}\n#SBATCH -p background\n#SBATCH -c 8\n\
#SBATCH --time=16:00:00\n#SBATCH --mem=32gb\n#SBATCH -o afit{0}-%j.out\n#SBATCH -e afit{0}-%j.err\n\nchmod a+x ALMA_Zfit.py\n\n\
python ALMA_Zfit.py {1} {2} {3}\n\
\nwait\n\nexit 0".format(idx, gals[i], lim1, lim2)]
    np.savetxt('../cluster_scripts/ALMA_Zfit_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
    idx +=1  

In [12]:
print(slrm_text[0])

#!/bin/bash
#SBATCH -J afit-11
#SBATCH -p background
#SBATCH -c 8
#SBATCH --time=16:00:00
#SBATCH --mem=32gb
#SBATCH -o afit11-%j.out
#SBATCH -e afit11-%j.err

chmod a+x ALMA_Zfit.py

python ALMA_Zfit.py 44042 11200 16500

wait

exit 0


In [2]:
########SF fit###########
SF_db = pd.read_pickle('../Casey_data/SF_fits/SF_db.pkl')

In [3]:
SF_db

,field,id,z,Z,Z_16,Z_84,t,t_16,t_84,tau,tau_16,tau_84,lmass,lmass_16,lmass_84,Av,Av_16,Av_84
0,GND,35075,1.780320,0.335,0.247,0.414,0.744,0.652,0.798,0.697,0.600,0.792,9.264000,9.233,9.294,0.213,0.176,0.265
1,GND,35204,2.096987,-99.000,0.000,-198.000,-99.000,0.000,-198.000,-99.000,0.000,-198.000,9.927095,0.000,-198.000,-99.000,0.000,-198.000
2,GND,28379,1.143402,0.054,0.054,0.135,2.279,1.995,2.611,1.129,0.837,1.495,10.543000,10.521,10.566,0.986,0.917,1.046
3,GND,32719,1.250495,1.551,0.809,1.630,0.858,0.707,1.133,0.711,0.464,1.157,9.574000,9.529,9.607,0.142,0.087,0.203
4,GND,30204,1.145387,-99.000,0.000,-198.000,-99.000,0.000,-198.000,-99.000,0.000,-198.000,10.191822,0.000,-198.000,-99.000,0.000,-198.000
5,GND,20801,1.999847,0.095,0.054,0.151,0.530,0.314,0.797,1.232,1.020,1.426,9.882000,9.769,9.940,0.579,0.509,0.716
6,GND,11839,1.240109,0.054,0.054,0.164,1.266,0.998,1.441,0.836,0.649,1.048,9.718000,9.680,9.747,0.500,0.408,0.587
7,GND,17927,1.346372,0.092,0.054,0.161,1.153,0.997,1.464,0.636,0.469,0.829,9.448000,9.420,9.476,0.321,0.223,0.384
8,GND,17324,2.304721,0.497,0.409,0.586,1.099,0.882,1.181,1.109,0.719,1.231,9.557000,9.519,9.590,0.248,0.207,0.289
9,GND,13777,1.231162,1.557,1.261,1.630,2.800,2.192,3.007,1.418,1.122,1.807,11.132000,11.076,11.169,1.467,1.348,1.599


In [6]:
idx = 1
for i in SF_db.index:      
    slrm_text= ["#!/bin/bash\n#SBATCH -J sfit-{0}\n#SBATCH -p background\n#SBATCH -c 8\n\
#SBATCH --time=16:00:00\n#SBATCH --mem=32gb\n#SBATCH -o sfit{0}-%j.out\n#SBATCH -e sfit{0}-%j.err\n\nchmod a+x SFfit.py\n\n\
python SFfit.py {1} {2} {3:1.5f} {4}\n\
\nwait\n\nexit 0".format(idx, SF_db.field[i], SF_db.id[i], SF_db.z[i], SF_db.lmass[i])]
    np.savetxt('../cluster_scripts/SFfit_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
    idx +=1  

In [7]:
print(slrm_text[0])

#!/bin/bash
#SBATCH -J sfit-69
#SBATCH -p background
#SBATCH -c 8
#SBATCH --time=16:00:00
#SBATCH --mem=32gb
#SBATCH -o sfit69-%j.out
#SBATCH -e sfit69-%j.err

chmod a+x SFfit.py

python SFfit.py GSD 40078 1.29567 9.506

wait

exit 0
